In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
# Read in data
train_data_path = "/kaggle/input/titanic/train.csv"
test_data_path = "/kaggle/input/titanic/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

train_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/titanic/train.csv'

In [5]:
# Find the missing data and process them
# mvc stands for missing value count
mvc_train = train_data.isnull().sum()
print(mvc_train)
mvc_test = test_data.isnull().sum()
print(mvc_test)

NameError: name 'train_data' is not defined

From the code above we know that the Age columns in both data frames are missing a lot of information. Instead of dropping them, we can fill them with the average age. We do the same for other columns with missing information.

In [4]:
# Data cleaninig
# Fill the missing Age with the average Age in train_data
train_age_mean = train_data.dropna(axis=0).Age.mean()
train_data.Age = train_data.Age.fillna(value = train_age_mean)
# Turn Sex into binary variate
gender = pd.get_dummies(train_data["Sex"])
train_data = pd.concat((train_data, gender), axis=1)
train_data = train_data.drop(["Sex", "female"], axis=1)
train_data = train_data.rename(columns={"male":"Sex"})

# Do the same for test_data
test_age_mean = test_data.dropna(axis=0).Age.mean()
test_data.Age = test_data.Age.fillna(value = test_age_mean)
test_gender = pd.get_dummies(test_data["Sex"])
test_data = pd.concat((test_data, test_gender), axis=1)
test_data = test_data.drop(["Sex", "female"], axis=1)
test_data = test_data.rename(columns={"male":"Sex"})

fare_mean = test_data.dropna(axis=0).Fare.mean()
test_data.Fare = test_data.Fare.fillna(value = fare_mean)

In [5]:
#Choose target and features
y = train_data.Survived
features = ['Age', 'Pclass', 'Sex', 'Fare', 'SibSp', 'Parch']
X = train_data[features]

# Train the model
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=0)

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state= 0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

best_tree_size = 10
min_mae = get_mae(10, train_X, val_X, train_y, val_y)
for node_number in range(10, 200, 10):
    cur_mae = get_mae(node_number, train_X, val_X, train_y, val_y)
    if (cur_mae < min_mae):
        min_mae = cur_mae
        best_tree_size = node_number
    
# Define & fit the model and make prediction
final_model = DecisionTreeRegressor(max_leaf_nodes = best_tree_size, random_state=0)
final_model.fit(X, y)
test_X = test_data[features]
predictions = final_model.predict(test_X)
predictions = [round(num) for num in predictions]

Save output.

In [6]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
